In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:

fig,ax=plt.subplots(1,3,figsize=(20,5))
sns.distplot(df['Age'],ax=ax[0])
sns.distplot(df['Annual Income (k$)'],ax=ax[1])
sns.distplot(df['Spending Score (1-100)'],ax=ax[2])

In [ ]:
px.violin(df,y='Annual Income (k$)',color='Gender',violinmode='overlay')
#most of women anuual income is concentrated is  40-80K, whereas men 50_80K

In [ ]:
px.box(df,x='Gender',y='Age')

In [ ]:
sns.pairplot(df)

In [ ]:
# we can see that spending score vs annual income there is cluster we can assume it as 5 clusters
px.scatter(df,x='Annual Income (k$)',y='Spending Score (1-100)',color='Age')


In [ ]:
df['Age_bins']=pd.qcut(df['Age'],q=3,labels=['young','middle_age','aged'])
px.scatter(df,x='Annual Income (k$)',y='Spending Score (1-100)',color='Age_bins')


In [ ]:
px.box(df,x='Age_bins',y='Annual Income (k$)')

In [ ]:
sns.barplot(df['Age_bins'],df['Spending Score (1-100)'],hue=df['Gender'])
#young people both men and female have high spending score than other as expected


In [ ]:
df['Gender']=df['Gender'].map({'Male':0,'Female':1})
df['Age_bins']=df['Age_bins'].map({'young':0,'middle_age':1,'aged':2})


In [ ]:
sns.heatmap(df.corr(),annot=True)

In [ ]:
from  sklearn.cluster import KMeans,hierarchical,DBSCAN
from sklearn.metrics  import silhouette_score

km_inertia=[]
X=df[['Age','Spending Score (1-100)','Annual Income (k$)']]
for n in range(1,10):
    km=KMeans(n_clusters=n,init='k-means++')
    km.fit(X)
    km_inertia.append(km.inertia_)


In [ ]:
sns.lineplot(x=np.arange(1,10),y=km_inertia,marker='o')#3 ,4,5 as k value

In [ ]:
km=KMeans(3)
df['k=3']=km.fit_predict(X)
km=KMeans(4)
df['k=4']=km.fit_predict(X)
km=KMeans(5)
df['k=5']=km.fit_predict(X)


In [ ]:
fig,ax=plt.subplots(1,3,figsize=(25,5))
sns.scatterplot(X['Annual Income (k$)'],X['Spending Score (1-100)'],hue=df['k=3'],ax=ax[0],palette=['red','green','blue'])
sns.scatterplot(X['Annual Income (k$)'],X['Spending Score (1-100)'],hue=df['k=4'],ax=ax[1],palette=['red','green','blue','yellow'])
sns.scatterplot(X['Annual Income (k$)'],X['Spending Score (1-100)'],hue=df['k=5'],ax=ax[2],palette=['red','green','blue','yellow','orange'])

In [ ]:
print(silhouette_score(X,labels=df['k=3']))
print(silhouette_score(X,labels=df['k=4']))
print(silhouette_score(X,labels=df['k=5']))


In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer
model = KMeans(5, random_state=42)
visualizer = SilhouetteVisualizer(model, colors='yellowbrick')
visualizer.fit(X)        # Fit the data to the visualizer
visualizer.show() 

#Hirerachical Clustering
Additive Clustering is bottom-up approach  ,here each datapoint is treated as seperate cluster in begining,and then twoclusters are merged based on 
the minimum distance between those  two. it continous till single clusters is left

In [ ]:

from sklearn.preprocessing import normalize

X_scaled=normalize(X)
X_scaled

In [ ]:
import scipy.cluster.hierarchy as shc


plt.figure(figsize=(20, 6))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(X, method='ward'))
plt.ylabel('EuclideanDistance')
plt.xlabel('Clusters')

#method ward groupsobs by reducing sum of squared distances of each observation from the average observation in a cluster. 

In [ ]:
from sklearn.cluster import AgglomerativeClustering

cluster=AgglomerativeClustering(n_clusters=5,linkage='ward')
X['hire_cluster']=cluster.fit_predict(X)

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(20,5))
sns.scatterplot(X['Spending Score (1-100)'],X['Annual Income (k$)'],hue=X['hire_cluster'],palette=['green','blue','orange','black','red'],ax=ax[0]).set_title('Hirerachical')

sns.scatterplot(X['Spending Score (1-100)'],X['Annual Income (k$)'],hue=df["k=5"],palette=['green','blue','orange','black','red'],ax=ax[1]).set_title('KMEANs at 5')

In [ ]:
print(silhouette_score(X,labels=X['hire_cluster']))#there is slight increase in score

In [ ]:
from sklearn.cluster import DBSCAN

epsilon = np.arange(8,12.75,0.25) # eps values to be investigated
no_samples = np.arange(3,10)

sil_score=[]
no_of_cluster=[]
for ep in epsilon:
    for sample in no_samples:
        db=DBSCAN(eps=ep,min_samples=sample)
        db.fit(X)
        no_of_cluster.append(len(np.unique(db.labels_)))
        print((ep,sample),silhouette_score(X,db.labels_),len(np.unique(db.labels_)))

In [ ]:
#for 6 cluster the score is high for eps=12.5,min_sample=4
db=DBSCAN(eps=12.5,min_samples=4).fit(X)
X['dbscan']=db.labels_

In [ ]:
fig,ax=plt.subplots(1,3,figsize=(25,5))

sns.scatterplot(y=X['Spending Score (1-100)'],x=X['Annual Income (k$)'],hue=X['hire_cluster'],palette=['green','blue','orange','black','red'],ax=ax[0]).set_title('Hirerachical')
sns.scatterplot(y=X['Spending Score (1-100)'],x=X['Annual Income (k$)'],hue=df["k=5"],palette=['green','blue','orange','black','red'],ax=ax[1]).set_title('KMEANs at 5')
sns.scatterplot(y=X['Spending Score (1-100)'],x=X['Annual Income (k$)'],hue=X["dbscan"],palette=['black','green','yellow','blue','orange','red'],ax=ax[2]).set_title('DBSCAN')

Based on Spending score and AnnualIncome  cluster are formed both in hireeachical and Kmean
1>low annual income(>40) and spending score than 40
2>annaul  income(>40)and spending score <60
3>high annual income(<60)and low spending score(>40)
4>high income and spending score
5>income between(30,60)and spending score (40,60)

Customer with high spending score and high income to be considered


In [ ]:
----------------Thanks ---------------